In [2]:
import pandas as pd
from pathlib import Path
from utils.distfit_scipy import *
import hydra
from omegaconf import DictConfig
import os

In [3]:
datapath = Path(Path().resolve().parents[2]) / 'data'
tpepath = datapath / 'forecast data' / 'TPE Search results'
tpepath.resolve()

WindowsPath('D:/Users/Heijden/Documents/PhD work/Current work/DR_IJmuiden/data/forecast data/TPE Search results')

In [4]:
waterboards = ['HHNK', 'HDSR', 'Rijnland', 'Waternet']
savepath = datapath / 'forecast data' / 'Distributions'

wbpath = tpepath / 'CQRDNN_Waterboards'
wlpath = tpepath / 'CQRDNN_WL_IJmuiden'
dampath = tpepath / 'CQRDNN_DAM'
quantiles=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

In [5]:
def read_forecasts(path):
    return pd.read_pickle(path / 'Forecasts' / 'test_fc_formatted.pkl').astype(float).round(3)#, index_col=0).astype(float)

In [10]:
# forecasts = {
#     wb: pd.read_csv(wbpath / wb / 'Forecasts' / 'test_fc_sorted.csv', index_col=0, parse_dates=True).astype(float) for wb in waterboards
# }

# forecasts = {
#     wb: pd.read_pickle(wbpath / wb / 'Forecasts' / 'test_fc_sorted.pkl') for wb in waterboards
# }
# forecasts['WL'] = pd.read_pickle(wlpath / 'Forecasts' / 'test_fc_sorted.pkl')

forecasts = {
    wb: read_forecasts(wbpath / wb) for wb in waterboards[:1]
}
forecasts['WL'] = read_forecasts(wlpath)
# forecasts['DAM'] = read_forecasts(dampath)
idx = pd.IndexSlice

In [9]:
forecasts['WL']

Aggregated
forecast_time             observation_time          quantile            
2019-01-01 00:00:00+00:00 2019-01-01 01:00:00+00:00 0.01          62.152
                                                    0.05          70.966
                                                    0.10          73.451
                                                    0.20          77.411
                                                    0.30          79.621
...                                                                  ...
2021-01-30 21:00:00+00:00 2021-02-01 21:00:00+00:00 0.70          23.413
                                                    0.80          28.420
                                                    0.90          41.535
                                                    0.95          55.684
                                                    0.99          99.738

[11395488 rows x 1 columns]

In [6]:
def reformat_df(df, colname):
    idx = pd.IndexSlice
    new_index = pd.MultiIndex.from_tuples([
        (ind[0], ind[1], q) for ind in df.index for q in quantiles
    ], names=['forecast_time', 'observation_time', 'quantile'])
    df_new = pd.DataFrame(index=new_index, columns=[colname])
    for ind in tqdm(df.index.get_level_values(0).unique()):
        df_new.loc[idx[ind, :, :], colname] = df.loc[idx[ind, :], :].values.flatten()
    return df_new

In [7]:
# for k in forecasts.keys():
#     forecasts[k] = reformat_df(forecasts[k], 'Aggregated')
#     if k in waterboards:
#         forecasts[k].to_pickle(wbpath / k / 'Forecasts' / 'test_fc_formatted.pkl')
#         forecasts[k].to_csv(wbpath / k / 'Forecasts' / 'test_fc_formatted.csv')
#     elif k == 'WL':
#         forecasts[k].to_pickle(wlpath / 'Forecasts' / 'test_fc_formatted.pkl')
        # forecasts[k].to_csv(wlpath / 'Forecasts' / 'test_fc_formatted.csv') 
        

In [8]:
# for k in forecasts.keys():
#     forecasts[k].index.names = ['forecast_time', 'observation_time', 'quantile']
#     if k in waterboards:
#         forecasts[k].to_pickle(wbpath / k / 'Forecasts' / 'test_fc_formatted.pkl')
#         forecasts[k].to_csv(wbpath / k / 'Forecasts' / 'test_fc_formatted.csv')
#     elif k == 'WL':
#         forecasts[k].to_pickle(wlpath / 'Forecasts' / 'test_fc_formatted.pkl')
#         forecasts[k].to_csv(wlpath / 'Forecasts' / 'test_fc_formatted.csv')

In [9]:
forecasts['HHNK']

Aggregated
forecast_time             observation_time          quantile            
2019-01-01 00:00:00+00:00 2019-01-01 01:00:00+00:00 0.01           5.743
                                                    0.05          10.636
                                                    0.10          12.951
                                                    0.20          14.989
                                                    0.30          16.368
...                                                                  ...
2021-01-30 19:00:00+00:00 2021-02-01 19:00:00+00:00 0.70          21.051
                                                    0.80          24.341
                                                    0.90          31.666
                                                    0.95          36.940
                                                    0.99          49.309

[11394240 rows x 1 columns]

In [10]:
quantiles=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

In [11]:
idx = pd.IndexSlice

In [12]:
ind = forecasts['HHNK'].index.get_level_values(0).unique()
forecasts['HHNK'].loc[idx[ind[0], ind[0]+pd.DateOffset(hours=3), :], 'Aggregated'].values.flatten()

array([ 0.368,  3.444,  5.996,  9.187, 11.623, 13.936, 15.789, 17.414,
       19.567, 22.01 , 24.936, 27.445, 33.237])

In [13]:
ind

DatetimeIndex(['2019-01-01 00:00:00+00:00', '2019-01-01 01:00:00+00:00',
               '2019-01-01 02:00:00+00:00', '2019-01-01 03:00:00+00:00',
               '2019-01-01 04:00:00+00:00', '2019-01-01 05:00:00+00:00',
               '2019-01-01 06:00:00+00:00', '2019-01-01 07:00:00+00:00',
               '2019-01-01 08:00:00+00:00', '2019-01-01 09:00:00+00:00',
               ...
               '2021-01-30 10:00:00+00:00', '2021-01-30 11:00:00+00:00',
               '2021-01-30 12:00:00+00:00', '2021-01-30 13:00:00+00:00',
               '2021-01-30 14:00:00+00:00', '2021-01-30 15:00:00+00:00',
               '2021-01-30 16:00:00+00:00', '2021-01-30 17:00:00+00:00',
               '2021-01-30 18:00:00+00:00', '2021-01-30 19:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='forecast_time', length=18260, freq=None)

In [14]:
# fitter = ScipyDistributionFitter(forecasts['HHNK'].iloc[:31*13*48], 'Aggregated', mask_regression_quantile=True, parallel=True)
fitter = ScipyDistributionFitter(forecasts['HHNK'], 'Aggregated', mask_regression_quantile=True, parallel=False)

  0%|          | 0/18260 [00:00<?, ?it/s]d:\Users\Heijden\Miniconda3\envs\scenario_generation\Lib\site-packages\scipy\stats\_continuous_distns.py:3783: RuntimeWarning: overflow encountered in exp
  return -sc.expm1(-np.exp(x))
d:\Users\Heijden\Miniconda3\envs\scenario_generation\Lib\site-packages\scipy\stats\_continuous_distns.py:3783: RuntimeWarning: overflow encountered in exp
  return -sc.expm1(-np.exp(x))
d:\Users\Heijden\Miniconda3\envs\scenario_generation\Lib\site-packages\scipy\stats\_continuous_distns.py:3783: RuntimeWarning: overflow encountered in exp
  return -sc.expm1(-np.exp(x))
d:\Users\Heijden\Miniconda3\envs\scenario_generation\Lib\site-packages\scipy\stats\_continuous_distns.py:3652: RuntimeWarning: overflow encountered in exp
  return np.exp(-np.exp(-x))
d:\Users\Heijden\Miniconda3\envs\scenario_generation\Lib\site-packages\scipy\stats\_continuous_distns.py:3783: RuntimeWarning: overflow encountered in exp
  return -sc.expm1(-np.exp(x))
d:\Users\Heijden\Miniconda3\env

In [17]:
fitter.fitted_distributions.index.get_level_values(0).unique()

DatetimeIndex(['2019-01-01 00:00:00+00:00', '2019-01-01 01:00:00+00:00',
               '2019-01-01 02:00:00+00:00', '2019-01-01 03:00:00+00:00',
               '2019-01-01 04:00:00+00:00', '2019-01-01 05:00:00+00:00',
               '2019-01-01 06:00:00+00:00', '2019-01-01 07:00:00+00:00',
               '2019-01-01 08:00:00+00:00', '2019-01-01 09:00:00+00:00',
               '2019-01-01 10:00:00+00:00', '2019-01-01 11:00:00+00:00',
               '2019-01-01 12:00:00+00:00', '2019-01-01 13:00:00+00:00',
               '2019-01-01 14:00:00+00:00', '2019-01-01 15:00:00+00:00',
               '2019-01-01 16:00:00+00:00', '2019-01-01 17:00:00+00:00',
               '2019-01-01 18:00:00+00:00', '2019-01-01 19:00:00+00:00',
               '2019-01-01 20:00:00+00:00', '2019-01-01 21:00:00+00:00',
               '2019-01-01 22:00:00+00:00', '2019-01-01 23:00:00+00:00',
               '2019-01-02 00:00:00+00:00', '2019-01-02 01:00:00+00:00',
               '2019-01-02 02:00:00+00:00', '2019-0

In [8]:
fitter.fitted_distributions

hour                           1                                           \
distribution                 dist                                  params   
2019-01-01 00:00:00+00:00  normal   [5.076823331813618, 2.98958533230256]   
2019-01-01 01:00:00+00:00  gumbel  [3.041544001020925, 4.010439544732075]   

hour                                       2   \
distribution                     KS      dist   
2019-01-01 00:00:00+00:00   0.03474  rayleigh   
2019-01-01 01:00:00+00:00  0.028776  rayleigh   

hour                                                                           \
distribution                                                 params        KS   
2019-01-01 00:00:00+00:00  [5.492697666368196, -1.6952084245563928]   0.03651   
2019-01-01 01:00:00+00:00  [5.595552233322425, -1.8091004705995406]  0.040923   

hour                              3   \
distribution                    dist   
2019-01-01 00:00:00+00:00  lognormal   
2019-01-01 01:00:00+00:00    frechet   

hour                                                                          \
distribution                                                          params   
2019-01-01 00:00:00+00:00  [5.6717449965440005, 5.782555819222319, -2.253...   
2019-01-01 01:00:00+00:00  [7.114978179942245, 2.5077556160036427, -4.619...   

hour                                          4   ...       45       46  \
distribution                     KS         dist  ...       KS     dist   
2019-01-01 00:00:00+00:00     0.045  weibull_min  ...    0.145  frechet   
2019-01-01 01:00:00+00:00  0.048429     rayleigh  ...  0.09297   gumbel   

hour                                                                          \
distribution                                                          params   
2019-01-01 00:00:00+00:00  [1.0901471748701688, 1.3235298655465035, -0.58...   
2019-01-01 01:00:00+00:00            [1.1618249753293346, 1.061278196870069]   

hour                                     47  \
distribution                     KS    dist   
2019-01-01 00:00:00+00:00     0.095  gumbel   
2019-01-01 01:00:00+00:00  0.072668  gumbel   

hour                                                                          \
distribution                                                params        KS   
2019-01-01 00:00:00+00:00   [1.20557322966015, 1.2949200524050954]  0.043537   
2019-01-01 01:00:00+00:00  [1.429931584395763, 1.6619567309724532]  0.030878   

hour                                    48  \
distribution                          dist   
2019-01-01 00:00:00+00:00  truncatednormal   
2019-01-01 01:00:00+00:00           gumbel   

hour                                                                          \
distribution                                                          params   
2019-01-01 00:00:00+00:00  [0.4999339387277897, 2.8261012244654893, 0.0, ...   
2019-01-01 01:00:00+00:00            [1.560357090304287, 1.9551984823207293]   

hour                                 
distribution                     KS  
2019-01-01 00:00:00+00:00  0.022947  
2019-01-01 01:00:00+00:00  0.033133  

[2 rows x 144 columns]

In [ ]:
distributions = {
    'normal': ot.Normal(),
    'lognormal': ot.LogNormal(),
    'gamma': ot.Gamma(),
    'beta': ot.Beta(),
    'gumbel': ot.Gumbel(),
    'frechet': ot.Frechet(), # inverted weibull
    'exponential': ot.Exponential(),
    'rayleigh': ot.Rayleigh(),
    'triangular': ot.Triangular(),
    'uniform': ot.Uniform(),
    'truncatednormal': ot.TruncatedNormal(),
    'pareto': ot.Pareto(),
    'weibull_min': ot.WeibullMin(),
    'weibull_max': ot.WeibullMax(),
    'logistic': ot.Logistic(),
}

In [ ]:
class DistributionFitter():
    def __init__(self, df, colname, quantiles=quantiles, distributions=distributions, mask_regression_quantile=False, parallel=True):
        self.df = df
        self.colname = colname
        self.quantiles = np.array(quantiles)
        self.distributions = distributions
        self.mask_regression_quantile = mask_regression_quantile
        
        if parallel:
            self.fit_distributions_parallel()
        else:
            self.fit_distributions()

    def get_quantiles(self, date, hour):
        regression_quantiles = self.df.loc[date, [f'{self.colname} +{hour} q{q}' for q in self.quantiles]].values
        if self.mask_regression_quantile:
            regression_quantiles[regression_quantiles<0] = np.nan
        return regression_quantiles
        
    def argmin_KS(self, parameters, dist_name, regression_quantiles):
        dist = self.distributions[dist_name]
        dist.setParameter(parameters)
        fitted_cdf = np.array([dist.computeCDF(q) for q in regression_quantiles[~np.isnan(regression_quantiles)]])
        return max(abs(fitted_cdf - self.quantiles[~np.isnan(regression_quantiles)]))
    
    def fit_distribution(self, regression_quantiles):
        results = {}
        for name, dist in self.distributions.items():
            try:
                to_minimize = partial(self.argmin_KS, dist_name=name, regression_quantiles=regression_quantiles)
                params = minimize(to_minimize, dist.getParameter(), method='Nelder-Mead', options={'maxiter': 1000}).x
                results[name] = {}
                results[name]['params'] = params
                results[name]['KS'] = self.argmin_KS(params, name, regression_quantiles)
            except:
                pass

        return results
    
    def get_best_distribution(self, date, hour):
        regression_quantiles = self.get_quantiles(date, hour)
        try:
            results = self.fit_distribution(regression_quantiles)
            best = min(results, key=lambda x: results[x]['KS'])
            return best, results[best]['params'], results[best]['KS']
        except ValueError:
            return np.nan, np.nan, np.nan
        
    
    def make_df_row(self, date):
        ret = pd.DataFrame(index=[date], columns=pd.MultiIndex.from_product([range(1,49), ['dist', 'params', 'KS']], names=['hour', 'distribution']))
        for hour in range(1,49):
            best, params, KS = self.get_best_distribution(date, hour)
            ret.loc[date, (hour, 'dist')] = best
            ret.loc[date, (hour, 'params')] = params
            ret.loc[date, (hour, 'KS')] = KS
        return ret
    
    def fit_distributions_parallel(self):
        num_cores = multiprocessing.cpu_count()
        results = Parallel(n_jobs=num_cores)(delayed(self.make_df_row)(date) for date in tqdm(self.df.index))
        self.fitted_distributions = pd.concat(results, axis=0).sort_index()
        return self.fitted_distributions
    
    def fit_distributions(self):
        self.fitted_distributions = pd.DataFrame(index=self.df.index, columns=pd.MultiIndex.from_product([range(1,49), ['dist', 'params', 'KS']], names=['hour', 'distribution']))
        for date in tqdm(self.df.index):
            for hour in range(1, 49):
                best, params, KS = self.get_best_distribution(date, hour)
                self.fitted_distributions.loc[date, (hour, 'dist')] = best
                self.fitted_distributions.loc[date, (hour, 'params')] = params
                self.fitted_distributions.loc[date, (hour, 'KS')] = KS
        return self.fitted_distributions


In [ ]:
fitter = DistributionFitter(forecasts['HHNK'], 'Aggregated', mask_regression_quantile=True)

In [ ]:
fitter.fitted_distributions.to_csv(savepath / 'HHNK.csv')

In [ ]:
for waterboard in waterboards:
    if (savepath / f'{waterboard}.csv').exists():
        continue
    fitter = DistributionFitter(forecasts[waterboard], 'Aggregated', mask_regression_quantile=True)
    fitter.fitted_distributions.to_csv(savepath / f'{waterboard}.csv')

if not (savepath / 'WL.csv').exists():
    fitter = DistributionFitter(forecasts['WL'], 'WL', mask_regression_quantile=False)
    fitter.fitted_distributions.to_csv(savepath / 'WL.csv')

In [ ]:
def plot_dists(df, dist_df, date):
    fig, ax = plt.subplots(4, 12, figsize=(20,10), sharey=True)
    for i, hour in enumerate(tqdm(range(1,49))):
        regression_quantiles = get_quantiles(df, date, hour, 'Aggregated')
        ax[i//12, i%12].plot(regression_quantiles, quantiles, '.')
        
        dist = dist_df.loc[date, (hour, 'dist')]
        params = dist_df.loc[date, (hour, 'params')]
        dist = distributions[dist]
        dist.setParameter(params)
        x = np.linspace(0, 1, 100)
        ax[i//12, i%12].plot(dist.computeQuantile(x), x)
        ax[i//12, i%12].set_title(f'{hour}:00')
    fig.tight_layout()
    
plot_dists(forecasts['HHNK'], dist_df, dist_df.index[0])

In [ ]:
fitter.fitted_distributions

In [ ]:
plot_dists(forecasts['HHNK'], dist_df, dist_df.index[0])

In [ ]:
plot_dists(forecasts['HHNK'], fitter.fitted_distributions, dist_df.index[0])

In [ ]:
df = forecasts['HHNK']

# distribution = fit_distribution_parallel(df, 'Aggregated')

# for dist in distributions.keys():
    
k='rayleigh'
regression_quantiles = get_quantiles(df, df.index[0], 1, 'Aggregated')
# quantile_values[quantile_values < 0] = 0

factory = ot.QuantileMatchingFactory(distributions[k], quantiles)
dist_estim = factory.buildFromQuantiles(regression_quantiles)
dist_estim

In [ ]:
dist_estim.computeCDF(np.array(regression_quantiles))

In [ ]:
res = fit_distribution(regression_quantiles)

In [ ]:
res['rayleigh']['dist'].getParameter()

In [ ]:
res

In [ ]:
fitted_q = dist_estim.computeQuantile(quantiles)
# fitted_q
# samples = dist_estim.computeCDF(quantile_values[0])
# samples

In [ ]:
fitted_q_values = dist_estim.computeQuantile(quantiles)
fitted_q_values = [q[0] for q in fitted_q_values]

fig, ax = plt.subplots()
ax.plot(fitted_q_values, quantiles, label='fit')
ax.plot(regression_quantiles, quantiles, label='reg')

ax.legend()

In [ ]:
fitted_q_values